In [1]:
## loading the data 
import pandas as pd
## defining the path 
PATH = "./data/clean_data.csv"
## loading 
df = pd.read_csv(PATH)
## reseting indeces
dfa = df.reset_index(drop=True)

## number of materials 
mats = [x for x in df.columns if "M-" in x]
n_mats = len(mats)
print(n_mats)

20


In [9]:
## current dataframe: Processes | Material 1| Material 2| Material 3| ... 
## convert to Processes | Materials | Values
long_df = pd.wide_to_long(dfa, ["M-"], i="Processes", j="Material").reset_index().rename(columns={"M-": "Consumption"})
## add Input or Output column
## x > 0 -> Input, x < 0 -> Output, x = 0 -> None
long_df['IO'] = long_df['Consumption'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0)
## adding the text to it in case
long_df['IO_txt'] = long_df['IO'].apply(lambda x: "Input" if x == 1 else "Output" if x == -1 else "None")
long_df

,Processes,Material,Consumption,IO,IO_txt
0,P-001,1,-73,-1,Output
1,P-002,1,0,0,None
2,P-003,1,0,0,None
3,P-004,1,0,0,None
4,P-005,1,0,0,None
...,...,...,...,...,...
595,P-026,20,90,1,Input
596,P-027,20,-83,-1,Output
597,P-028,20,-22,-1,Output
598,P-029,20,16,1,Input


In [10]:
## the dataframe is per process, 
long_df = long_df.sort_values(by=['Processes','Material'])
long_df_nz = long_df.query("Consumption != 0")


In [12]:
long_df

,Processes,Material,Consumption,IO,IO_txt
0,P-001,1,-73,-1,Output
30,P-001,2,0,0,None
60,P-001,3,59,1,Input
90,P-001,4,0,0,None
120,P-001,5,48,1,Input
...,...,...,...,...,...
479,P-030,16,-98,-1,Output
509,P-030,17,0,0,None
539,P-030,18,0,0,None
569,P-030,19,33,1,Input


In [56]:
## Used Materials
input_materials = long_df_nz.query("IO == 1")
used_materials_per_process = input_materials.groupby("Processes")['Material'].apply(list).reset_index()
used_materials_per_process.rename(columns={"Material": "Inputs"}, inplace=True)
## Energy to Input 
used_consumption_materials_per_process = input_materials.groupby("Processes")['Consumption'].apply(list).reset_index()
used_consumption_materials_per_process.rename(columns={"Consumption": "InputCost"}, inplace=True)
## merge 
#used_materials_df = input_materials.merge(used_consumption_materials_per_process, on="Processes", how="left")
#used_materials_df = used_materials_df.merge(used_consumption_materials_per_process, on="Processes", how="left")
#used_materials_df
used_materials_per_process

,Processes,Inputs
0,P-001,"[3, 5]"
1,P-002,"[5, 8, 10, 11, 13, 15, 17]"
2,P-003,"[4, 5, 8, 9, 13, 15]"
3,P-004,"[2, 7, 18]"
4,P-005,"[5, 8, 9, 12, 19, 20]"
5,P-006,"[7, 13, 15, 19, 20]"
6,P-007,"[1, 6, 7, 8, 9, 10, 19]"
7,P-008,"[1, 2, 5, 6, 16, 19]"
8,P-009,"[1, 2, 5, 12, 17]"
9,P-010,"[2, 3, 8, 10, 15]"


In [44]:


## Produced materials
output_materials = long_df_nz.query("IO == -1")
produced_materials_per_process = output_materials.groupby("Processes")['Material'].apply(list).reset_index()
produced_materials_per_process.rename(columns={"Material": "Outputs"}, inplace=True)
## Consumption Needed
produced_consumption_materials_per_process = output_materials.groupby("Processes")['Consumption'].apply(list).reset_index()
produced_consumption_materials_per_process.rename(columns={"Consumption": "OutputCost"}, inplace=True)
## merge 
produced_materials_df = output_materials.merge(produced_materials_per_process, on="Processes", how="left")
produced_materials_df = produced_materials_df.merge(produced_consumption_materials_per_process, on="Processes", how="left")


In [42]:
used_materials_df

,Processes,Material,Consumption,IO,IO_txt,InputCost
0,P-001,3,59,1,Input,"[59, 48]"
1,P-001,5,48,1,Input,"[59, 48]"
2,P-002,5,73,1,Input,"[73, 11, 69, 71, 36, 33, 64]"
3,P-002,8,11,1,Input,"[73, 11, 69, 71, 36, 33, 64]"
4,P-002,10,69,1,Input,"[73, 11, 69, 71, 36, 33, 64]"
...,...,...,...,...,...,...
155,P-029,14,36,1,Input,"[73, 69, 88, 36, 88, 16]"
156,P-029,15,88,1,Input,"[73, 69, 88, 36, 88, 16]"
157,P-029,20,16,1,Input,"[73, 69, 88, 36, 88, 16]"
158,P-030,12,43,1,Input,"[43, 33]"


In [39]:
produced_materials_df

,Processes,Material,Consumption,IO,IO_txt,Outputs,OutputCost
0,P-001,1,-73,-1,Output,"[1, 12, 15, 18, 19]","[-73, -60, -26, -9, -15]"
1,P-001,12,-60,-1,Output,"[1, 12, 15, 18, 19]","[-73, -60, -26, -9, -15]"
2,P-001,15,-26,-1,Output,"[1, 12, 15, 18, 19]","[-73, -60, -26, -9, -15]"
3,P-001,18,-9,-1,Output,"[1, 12, 15, 18, 19]","[-73, -60, -26, -9, -15]"
4,P-001,19,-15,-1,Output,"[1, 12, 15, 18, 19]","[-73, -60, -26, -9, -15]"
...,...,...,...,...,...,...,...
132,P-030,6,-40,-1,Output,"[4, 6, 8, 11, 16, 20]","[-16, -40, -5, -4, -98, -28]"
133,P-030,8,-5,-1,Output,"[4, 6, 8, 11, 16, 20]","[-16, -40, -5, -4, -98, -28]"
134,P-030,11,-4,-1,Output,"[4, 6, 8, 11, 16, 20]","[-16, -40, -5, -4, -98, -28]"
135,P-030,16,-98,-1,Output,"[4, 6, 8, 11, 16, 20]","[-16, -40, -5, -4, -98, -28]"


In [31]:
materials = long_df_nz.groupby("Processes")['Material'].apply(list).reset_index().rename(columns={"Material": "Inputs/Outputs"})
costs = long_df_nz.groupby("Processes")['Consumption'].apply(list).reset_index().rename(columns={"Consumption": "Costs"})
## merge the dataframes on processes 
materials_costs = materials.merge(costs, on="Processes", how="left")
## merge long_df_nz with the materials_costs
final_df = long_df_nz.merge(materials_costs, on="Processes", how="left")
final_df = final_df.drop_duplicates("Processes").drop(["Material","Consumption","IO","IO_txt",],axis=1)
## sum only the positive values in the list 
output_cost = lambda x: x if x>0 else 0
input_cost = lambda x: x if x<0 else 0
## map the function to the list
final_df['OutputCost'] = final_df['Costs'].apply(lambda x: sum(list(map(output_cost, x))))
final_df['InputCost'] = final_df['Costs'].apply(lambda x: sum(list(map(input_cost, x))))
final_df

,Processes,Outputs,Costs,OutputCost,InputCost
0,P-001,"[1, 3, 5, 12, 15, 18, 19]","[-73, 59, 48, -60, -26, -9, -15]",107,-183
7,P-002,"[5, 8, 10, 11, 13, 14, 15, 16, 17, 19, 20]","[73, 11, 69, 71, 36, -33, 33, -82, 64, -38, -92]",357,-245
18,P-003,"[4, 5, 8, 9, 12, 13, 14, 15]","[58, 26, 92, 43, -42, 34, -4, 5]",258,-46
26,P-004,"[2, 3, 5, 7, 8, 11, 13, 16, 18]","[71, -28, -76, 4, -12, -11, -21, -28, 18]",93,-176
35,P-005,"[2, 4, 5, 8, 9, 10, 11, 12, 13, 18, 19, 20]","[-2, -71, 37, 19, 16, -43, -9, 98, -7, -91, 22...",196,-223
47,P-006,"[2, 7, 9, 13, 15, 19, 20]","[-97, 79, -11, 8, 16, 20, 22]",145,-108
54,P-007,"[1, 6, 7, 8, 9, 10, 12, 19]","[87, 4, 22, 60, 69, 59, -96, 1]",302,-96
62,P-008,"[1, 2, 4, 5, 6, 8, 16, 17, 19, 20]","[47, 10, -28, 6, 58, -5, 74, -71, 18, -33]",213,-137
72,P-009,"[1, 2, 5, 6, 7, 8, 11, 12, 16, 17]","[25, 51, 67, -5, -95, -34, -52, 80, -50, 24]",247,-236
82,P-010,"[2, 3, 6, 8, 10, 12, 15, 18, 20]","[22, 15, -15, 64, 71, -87, 64, -16, -70]",236,-188


In [15]:
# long_df_nz['Material'] = long_df_nz.groupby("Processes")['Material'].apply(list).reset_index()
# long_df_nz['Consumption'] = long_df_nz.groupby("Processes")['Consumption'].apply(list).reset_index()
# long_df_nz

ValueError: Columns must be same length as key

In [ ]:
used_materials_df

In [ ]:
from neo4j import GraphDatabase

In [ ]:
## create a class to make the connection to the database 
class DataBaseConnector: 
    
    ## constructor 
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    ## close function 
    def close(self):
        self.driver.close()
        

        

## it should have the following methods: 

## insert a new process with associated materials and information 

## insert a new material with associated information

## delete a process

## delete a material 

## replate a process with a new one

## replate a material with a new one

In [ ]:
!pip install -qq neomodel

Essentially doing this 

1) Database 
2) Connection to DB with action 
3) Ask queries to the system

In [ ]:
def make_connection_url(url, username, password):
    """Returns the connection URL and the driver."""
    return f"neo4j://{username}:{password}@{url}"

In [ ]:
## use neo4j-python to connect to the database
from neomodel import db 
## make the url to connect 
base_url  = "localhost:7687"
username = 'neo4j' ## 
password = "CircularEconomyDB" ## 
url = make_connection_url(base_url, username, password)

## make the connection 
db.set_connection(url)

In [ ]:
## The database will have: Processes and Materials 

## The base is a Material: 
## a material has: name, quantity, unit, price, input/output, process and description 


## the base Process is: 
## a process has: name, input/output,  materials and description



In [ ]:
##- input: materials required to make the current materias
##- associated process: processes that use the material

## QUERIES NEED TO BE MADE TO POPULATE THE ABOVE 

In [ ]:
## create the classes of Materials and Processes 

from neomodel import (StructuredNode, StringProperty, IntegerProperty, RelationshipTo)

## define the class for the materials

class Material(StructuredNode):
    """
    A Material is a substance that is used in a process.
    
    It contains the following information: 
        
        - id: unique identifier of the material  
        - name: the name of the material
        - quantity: the quantity of the material
        - unit: the unit which the quantity is measured
        - cost: the price of the material
        - description (optional): a description of the material
    """
    uuid = StringProperty(unique_index=True)
    name = StringProperty()
    quantity = IntegerProperty()
    unit = StringProperty()
    cost = IntegerProperty()
    description = StringProperty() 


In [ ]:
## define the class for the processes
class Process(StructuredNode):
    """
    A Process is a process that is used to produce a material.
    
    It contains the following information: 
        
        - uuid: unique identifier of the process  
        - name: the name of the process
        - materials used in the process (input/output): a list of materials
        - output of the process: the material(s) produced
        - description (optional): a description of the process
    """
    uuid = StringProperty(unique_index=True)
    name = StringProperty()
    materials_used = RelationshipTo('Material', 'USED_IN')
    output_of_process = RelationshipTo('Material', 'OUTPUT_OF')
    description = StringProperty()   

In [ ]:
material_dict = {"uuid": "ee",
                 "name": "Material 1",
                 "quantity": 1, "unit": "kg",
                 "cost": 1,
                 "description":"Material 1 - is made of SodiumChloride (Na) with a concentration of 0.1%"} ## the dictionary for the first material

mat = Material(**material_dict)
mat.save()

In [ ]:
#driver = GraphDatabase.driver(URL, auth=(username, password))
# neo_url = "http://localhost:7474"
# driver = GraphDatabase.driver(neo_url, auth=(username, password))

# with driver.session() as sess:
    
    
    

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(URL, auth=(username, password))

def create_friend_of(tx, name, friend):
    tx.run("MATCH (a:Person) WHERE a.name = $name "
           "CREATE (a)-[:KNOWS]->(:Person {name: $friend})",
           name=name, friend=friend)

with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Bob")

with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Carl")

driver.close()